# IonQ Boostvqe demo

In this notebook, we demonstrate how to run boostvqe on IonQ emulators (similarly hardwarws with some backend modifications).

We are following on the DBQA GCI circuit synthesis notebook, which can be found following the link: https://github.com/qiboteam/boostvqe/blob/quantinuum_boostvqe-demo/notebooks/quantinuum_demo/gci_boostvqe_circuit_synthesis.ipynb

It supplies a VQE and a GCI circuits in QASM format which we will load our circuits from.

## Step 1: Load circuits

In [66]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qibo import hamiltonians
import matplotlib.pyplot as plt
from ionq_utils import *
from scipy.linalg import eigh

In [17]:
# file paths
vqe_circ_file = "results/vqe_circ_4.qasm"
gci_circ_file = "results/gci_circ_4.qasm"

In [18]:
vqe_circ = QuantumCircuit.from_qasm_file(vqe_circ_file)
# render_circuit_jupyter(vqe_circ)

In [19]:
gci_circ = QuantumCircuit.from_qasm_file(gci_circ_file)
# render_circuit_jupyter(gci_circ)

### Check analytical energy

In this step we are making sure that the correct circuits have been imported.

In [67]:
# create target hamiltonian in qiskit
ham = xxz_hamiltonian(4, 0.5)
ham_matrix = ham.to_matrix()
eigenvalues, eigenstates = eigh(ham_matrix)
target_energy = eigenvalues[0]

In [68]:
state_vec_vqe = Statevector.from_instruction(vqe_circ)
expectation_value_vqe = np.real(np.dot(state_vec_vqe.data.conj().T, np.dot(ham_matrix, state_vec_vqe.data)))
state_vec_gci = Statevector.from_instruction(gci_circ)
expectation_value_gci = np.real(np.dot(state_vec_gci.data.conj().T, np.dot(ham_matrix, state_vec_gci.data)))
print("Target energy:", target_energy)
print("VQE energy:", expectation_value_vqe)
print("GCI energy:", expectation_value_gci)

Target energy: -6.744562646538019
VQE energy: -6.309812021233839
GCI energy: -6.741347828261663


## Step 2: Connect to IonQ server

In [22]:
from qiskit_ionq import IonQProvider
from copy import deepcopy

my_api_key = "pOiUVlzriOoF2wX1kp3lIqid1OMhwXZ5"
provider = IonQProvider(my_api_key)
ionq_backend = provider.get_backend("ionq_simulator")

/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for simulator: 'qubits'. Defaulting to 100.
  warnings.warn(
/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for ionq_qpu: 'qubits'. Defaulting to 100.
  warnings.warn(
/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for ionq_simulator: 'qubits'. Defaulting to 100.
  warnings.warn(


## Step 3: Prepare measurement circuits

In this step, we prepare a list of measurement circuits to measure the energy expectation of the Hamiltonian.
Since our example uses XXZ model, we will need to measure the circuits in 3 separate bases:
1. X
2. Y
3. Z

In [ ]:
from pytket.extensions.qiskit import tk_to_qiskit
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit
from pytket.utils.operators import QubitPauliOperator

In [45]:
meas_vqe_circ_list = rotate_circuit_XYZ(vqe_circ)
meas_gci_circ_list = rotate_circuit_XYZ(gci_circ)

## Step 4: Execute circuits


In [58]:
zz = xxz_hamiltonian(4,1,select='ZZ').to_matrix()
coef = [1,1,0.5]

In [59]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(meas_vqe_circ_list):
    job = ionq_backend.run(qc_m, shots=shots)
    result = job.result()
    distribution = result.get_probabilities()
    expectation = sample_to_expectation(zz, distribution) * coef[i]
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    
vqe_expect = xxz_expectation
print('Expectation energy:', vqe_expect)

XX -3.0812240639999997
YY -1.967248336
ZZ -1.272256812
Expectation energy: -6.320729212


In [76]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(meas_gci_circ_list):
    job = ionq_backend.run(qc_m, shots=shots)
    result = job.result()
    distribution = result.get_probabilities()
    expectation = sample_to_expectation(zz, distribution) * coef[i]
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    
gci_expect = xxz_expectation
print('Expectation energy:', gci_expect)

XX -2.7795153800000003
YY -2.7820648959999996
ZZ -1.180009208
Expectation energy: -6.741589484


## Step 5: Run with noise

In [61]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(meas_vqe_circ_list):
    job = ionq_backend.run(qc_m, shots=shots, noise_model='aria-1')
    result = job.result()
    distribution = result.get_probabilities()
    expectation = sample_to_expectation(zz, distribution) * coef[i]
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    
vqe_expect_noise = xxz_expectation
print('Expectation energy:', vqe_expect_noise)

XX -3.028
YY -1.784
ZZ -1.2859999999999998
Expectation energy: -6.098


In [77]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(meas_gci_circ_list):
    job = ionq_backend.run(qc_m, shots=shots, noise_model='aria-1')
    result = job.result()
    distribution = result.get_probabilities()
    expectation = sample_to_expectation(zz, distribution) * coef[i]
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    
gci_expect_noise = xxz_expectation
print('Expectation energy:', gci_expect_noise)

XX -0.616
YY -0.44800000000000006
ZZ -0.31800000000000006
Expectation energy: -1.3820000000000001


## Report

In [78]:
def report_ionq(vqe_analytical, gci_analytical, ham_matrix, expval_vqe, expval_gci, expval_vqe_noise, expval_gci_noise):
    eigenvalues, eigenstates = eigh(ham_matrix)
    ground_state_energy = eigenvalues[0]
    vqe_energy = vqe_analytical
    gci_energy = gci_analytical
    gap = float(eigenvalues[1] - eigenvalues[0])
    return (
        dict(
            nqubits = int(np.log(len(ham_matrix))/np.log(2)),
            gci_energy=float(gci_energy),
            vqe_energy=float(vqe_energy),
            vqe_energy_emulator=float(expval_vqe),
            gci_energy_emulator=float(expval_gci),
            vqe_energy_emulator_noise=float(expval_vqe_noise),
            gci_energy_emulator_noise=float(expval_gci_noise),
            target_energy=ground_state_energy,
            diff_vqe_target=vqe_energy - ground_state_energy,
            diff_gci_target=gci_energy - ground_state_energy,
            diff_vqe_target_emulator=expval_vqe - ground_state_energy,
            diff_gci_target_emulator=expval_gci - ground_state_energy,
            diff_vqe_target_emulator_noise=expval_vqe_noise - ground_state_energy,
            diff_gci_target_emulator_noise=expval_gci_noise - ground_state_energy,
            gap=gap,
            diff_vqe_target_perc=abs(vqe_energy - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            diff_gci_target_perc=abs(gci_energy - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            diff_vqe_target_perc_emulator=abs(expval_vqe - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            diff_gci_target_perc_emulator=abs(expval_gci - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            diff_vqe_target_perc_emulator_noise=abs(expval_vqe_noise - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            diff_gci_target_perc_emulator_noise=abs(expval_gci_noise - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            fidelity_witness_vqe=1 - (vqe_energy - ground_state_energy) / gap,
            fidelity_witness_gci=1 - (gci_energy - ground_state_energy) / gap,
            fidelity_witness_vqe_emulator=1 - (expval_vqe - ground_state_energy) / gap,
            fidelity_witness_gci_emulator=1 - (expval_gci - ground_state_energy) / gap,
            fidelity_witness_vqe_emulator_noise=1 - (expval_vqe_noise - ground_state_energy) / gap,
            fidelity_witness_gci_emulator_noise=1 - (expval_gci_noise - ground_state_energy) / gap,
        )
    )

In [79]:
report = report_ionq(expectation_value_vqe, expectation_value_gci, ham_matrix, vqe_expect, gci_expect, vqe_expect_noise, gci_expect_noise)

In [80]:
# Creating a DataFrame for the table
import pandas as pd
df = pd.DataFrame({
    "Analytical": [
        report['vqe_energy'],
        report['gci_energy'],
        report['diff_vqe_target'],
        report['diff_gci_target'],
        report['diff_vqe_target_perc'],
        report['diff_gci_target_perc'],
        report['fidelity_witness_vqe'],
        report['fidelity_witness_gci']
    ],
    "Emulator": [
        report['vqe_energy_emulator'],
        report['gci_energy_emulator'],
        report['diff_vqe_target_emulator'],
        report['diff_gci_target_emulator'],
        report['diff_vqe_target_perc_emulator'],
        report['diff_gci_target_perc_emulator'],
        report['fidelity_witness_vqe_emulator'],
        report['fidelity_witness_gci_emulator']
    ],
    "Emulator with Noise": [
        report['vqe_energy_emulator_noise'],
        report['gci_energy_emulator_noise'],
        report['diff_vqe_target_emulator_noise'],
        report['diff_gci_target_emulator_noise'],
        report['diff_vqe_target_perc_emulator_noise'],
        report['diff_gci_target_perc_emulator_noise'],
        report['fidelity_witness_vqe_emulator_noise'],
        report['fidelity_witness_gci_emulator_noise']
    ]
}, index=[
    "VQE energy",
    "GCI energy",
    "Difference to target (VQE)",
    "Difference to target (GCI)",
    "Percentage difference to target (VQE)",
    "Percentage difference to target (GCI)",
    "Fidelity witness (VQE)",
    "Fidelity witness (GCI)"
])

# Display the DataFrame
df

,Analytical,Emulator,Emulator with Noise
VQE energy,-6.309812,-6.320729,-6.098000
GCI energy,-6.741348,-6.741589,-1.382000
Difference to target (VQE),0.434751,0.423833,0.646563
Difference to target (GCI),0.003215,0.002973,5.362563
Percentage difference to target (VQE),6.445942,6.284076,9.586428
Percentage difference to target (GCI),0.047665,0.044082,79.509420
Fidelity witness (VQE),0.841596,0.845573,0.764421
Fidelity witness (GCI),0.998829,0.998917,-0.953886
